# Map-Making using Planck Acquisition

In this notebook, we describe how to make the map-making process using Planck acquisition to correct edge effects. We will merge QUBIC and Planck acquisition to benefit of both instrument, QUBIC for the Q and U sensitivity and Planck for the large coverage and sensitivity on I.

In [ ]:
from __future__ import division
from pyoperators import pcg
from pysimulators import profile

# QUBIC packages
import qubic
from qubicpack.utilities import Qubic_DataDir
from qubic.data import PATH
from qubic.io import read_map
from qubic import QubicSkySim as qss
import Acquisition as Acq

# Display packages
import healpy as hp
import matplotlib.pyplot as plt

# General packages
import numpy as np
import pysm3
import warnings
from qubic import QubicSkySim as qss
import pysm3.units as u
from importlib import reload
from pysm3 import utils

# PyOperators packages
from pyoperators import (
    BlockColumnOperator, BlockDiagonalOperator, BlockRowOperator,
    CompositionOperator, DiagonalOperator, I, IdentityOperator,
    MPIDistributionIdentityOperator, MPI, proxy_group, ReshapeOperator,
    rule_manager, pcg, Operator)
warnings.filterwarnings("ignore")
%matplotlib inline

# %config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# QUBIC dictionary

We first import the QUBIC dictionary to define general stuff like pixelization, seen region on the sky, etc... 

In [ ]:
# Repository for dictionary
global_dir = '/pbs/home/m/mregnier/Libs/qubic/qubic/'#Qubic_DataDir()
print(global_dir)
dictfilename = global_dir + 'dicts/pipeline_demo.dict'

# Read dictionary chosen
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)

from qubic import SpectroImLib as sp
from importlib import reload

Nf = 1
relative_bandwidth = 0.25
Nbfreq, nus_edge, nus, deltas, Delta, Nbbands = qubic.compute_freq(150, Nfreq=Nf)
print(nus)
print(nus_edge)

d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
d['nf_recon'] = Nf
d['nf_sub'] = Nf
d['nside'] = 256
npix=12*d['nside']**2
d['RA_center'] = 0
d['DEC_center'] = -57
center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
d['effective_duration'] = 4
#d['dtheta'] = 15
d['npointings'] = 1000
d['tol'] = 5e-4
d['filter_nu'] = 150e9
d['photon_noise'] = False
d['noiseless'] = False
d['config'] = 'FI'
d['MultiBand'] = False
d['planck'] = True

Then, we define specific qubic package for Map-Making. The Pointing package define the information on the scanning strategy (random pointing here). The Scene package define things realted to the pixelization. The Instrument package define the QUBIC instrument forexample for systematics effect. Then we define the acquisition for our instrument.

In [ ]:
Qubic_sky = qss.Qubic_sky({'cmb':42}, d)
mapin = Qubic_sky.get_simple_sky_map()

# Pointing
p = qubic.get_pointing(d)

# Scene
s = qubic.QubicScene(d)

# Instrument
q = qubic.QubicInstrument(d)

# QUBIC Acquisition
qubic_acquisition = Acq.QubicAcquisition(q, p, s, d)

The acquisition allows you to define the QUBIC operator which tak eas input a sky map with shape ($N_{pix}$, $N_{stk}$) and returns TOD after many rotations and transformations. For the next notebook, it will be very inportant to understand the structure of this operator... To be breaf, this operator can be seen as a list of function.

In [ ]:
H_qubic = qubic_acquisition.get_operator()
H_qubic

Here we define $N^{-1}$ the inverse noise covariance matrix in time domain.

In [ ]:
invntt = qubic_acquisition.get_invntt_operator()

As we said before, we compute our observations using QUBIC operator.

In [ ]:
tod = qubic_acquisition.get_observation(map=mapin[0], convolution=False, noiseless=False)

plt.figure(figsize=(15, 5))
plt.plot(tod[0])
plt.show()

In a very generic way, all of this work is done to solve this equation :

$$y = (H^t N^{-1} H)^{-1} H^t N^{-1} d$$

where d is your data and $y$ is the observed sky. In practical, we can not solve that analytical due to the very large size of $H$ and $N^{-1}$, then the PCG is here to solve iteratively. We just define here both side of this equation like : 

$$A = (H^t N^{-1} H)$$
$$b = H^t N^{-1} d$$

In [ ]:
A = H_qubic.T * invntt * H_qubic
b = H_qubic.T * invntt * tod

The PCG (Preconditionned Conjugated Gradient) solve the above equation iteratively, the convergence of the method is conditionnned by $M$ which is called the preconditionner. For QUBIC only acquisition, it can be define as function of the coverage.

In [ ]:
def get_preconditioner(cov):
    if cov is not None:
        cov_inv = 1 / cov
        cov_inv[np.isinf(cov_inv)] = 0.
        preconditioner = DiagonalOperator(cov_inv, broadcast='rightward')
    else:
        preconditioner = None
    return preconditioner

cov = qubic_acquisition.get_coverage()
pixok = cov > 0
mapin[0, ~pixok, :] = 0
M = get_preconditioner(cov)

Here we call the PCG from the PyOperators package, we have to mentionned a tolerance.

In [ ]:
tol=5e-4

solution_qubic = pcg(A, b, x0=None, M=M, tol=tol, disp=True)

Now we can see the result of the convergence...

In [ ]:
def display_maps(inputs, outputs, display=True, rot=None, res=None):
    
    rI = inputs[:, 0] - outputs[:, 0]
    rQ = inputs[:, 1] - outputs[:, 1]
    rU = inputs[:, 2] - outputs[:, 2]
    r=np.array([rI, rQ, rU])
    
    
    plt.figure(figsize=(10, 10))
    
    stk=['I', 'Q', 'U']
    k=1
    for i in range(3):
        if i == 0: min, max = -300, 300
        else: min, max = -8, 8
        hp.gnomview(inputs[:, i], rot=rot, reso=res, cmap='jet', min=min, max=max, sub=(3, 3, k), title='Input - {}'.format(stk[i]))
        k+=1
        hp.gnomview(outputs[:, i], rot=rot, reso=res, cmap='jet', min=min, max=max, sub=(3, 3, k), title='Output - {}'.format(stk[i]))
        k+=1
        hp.gnomview(r[i], rot=rot, reso=res, cmap='jet', min=min, max=max, sub=(3, 3, k), title='Residual - {}'.format(stk[i]))
        k+=1
    
    plt.show()
    
    return r

r=display_maps(mapin[0], solution_qubic['x'], display=True, rot=center, res=25)

In the inner part of the coverage, the result is quite good. It seems that the PCG solve correctly the Map-Making equation and so converge to the solution. Unfortunately, on the edge of the coverage hte reconstruction is very bad and the error increase a lot... Missing data is the cause and a way to solve this problem is to merge the QUBIC acquisition with the planck aquisition which have seen the whole sky.

We define here the PlanckAcquisition which needs the central frequency that you are using (150 or 220 GHz) and the same scene as before.

In [ ]:
# Planck Acquisition
reload(Acq)
Qubic_sky = qss.Qubic_sky({'cmb':42}, d)
mapin = Qubic_sky.get_simple_sky_map()

planck_acquisition = Acq.PlanckAcquisition(143, s, true_sky=mapin[0], mask=None)

In [ ]:
# Merge both acquisition
qubicplanck_acquisition = Acq.QubicPlanckAcquisition(qubic_acquisition, planck_acquisition)

Here, we define the QUBIC-Planck Operator. As before, look at this operator is very important for next steps.

In [ ]:
H = qubicplanck_acquisition.get_operator()
H

In [ ]:
invntt = qubicplanck_acquisition.get_invntt_operator()
invntt

Here, we take a look of data. The first part of the TOD is the QUBIC part and the second is the sky seen by Planck.

In [ ]:
#tod = qubicplanck_acquisition.get_observation(convolution=False, noiseless=False)
n = qubicplanck_acquisition.get_noise()
tod = H(mapin[0]) + n
plt.figure(figsize=(15, 5))
plt.plot(tod)
plt.show()

In [ ]:
A = H.T * invntt * H
b = H.T * invntt * tod

Here, the preconditionner is not a function of the QUBIC coverage because Planck acquisition is turn on. The simplest preconditionner is the entire sky itself.

In [ ]:
M = Acq.get_preconditioner(np.ones(12*d['nside']**2))

In [ ]:
tol=1e-4

solution_qubic_planck = pcg(A, b, x0=None, M=M, tol=tol, disp=True)

You can note that the convergence is much faster than before with the QUBIC only acquisition. This is due to the fact that edge effects have disappeared ! Let's take a look of reconstructed maps.

In [ ]:
def display_maps(inputs, outputs, display=True, rot=None, res=None):
    
    rI = inputs[:, 0] - outputs[:, 0]
    rQ = inputs[:, 1] - outputs[:, 1]
    rU = inputs[:, 2] - outputs[:, 2]
    r=np.array([rI, rQ, rU])
    
    
    plt.figure(figsize=(10, 10))
    
    stk=['I', 'Q', 'U']
    k=1
    for i in range(3):
        if i == 0: min, max = -300, 300
        else: min, max = -8, 8
        hp.gnomview(inputs[:, i], rot=rot, reso=res, cmap='jet', min=min, max=max, sub=(3, 3, k), title='Input - {}'.format(stk[i]))
        k+=1
        hp.gnomview(outputs[:, i], rot=rot, reso=res, cmap='jet', min=min, max=max, sub=(3, 3, k), title='Output - {}'.format(stk[i]))
        k+=1
        hp.gnomview(r[i], rot=rot, reso=res, cmap='jet', min=min, max=max, sub=(3, 3, k), title='Residual - {}'.format(stk[i]))
        k+=1
    
    plt.show()
    
    return r

rpl=display_maps(mapin[0], solution_qubic_planck['x'], display=True, rot=center, res=25)

We have now reconstructed maps which combine QUBIC and Planck sensitivity ! Let's take a look of profiles now.

In [ ]:
xx, yyI, yyQ, yyU = qss.get_angular_profile(np.array([r[0], r[1], r[2]]).T, nbins=30, separate=True, center=center, thmax=30)
xx_pl, yyI_pl, yyQ_pl, yyU_pl = qss.get_angular_profile(np.array([rpl[0], rpl[1], rpl[2]]).T, nbins=30, separate=True, center=center, thmax=50)

In [ ]:
covnorm = cov / np.max(cov)
a_planck=qubic.PlanckAcquisition(nus[0], s, true_sky=mapin[0], mask=None)
noise_planck=a_planck.get_noise()

plt.figure(figsize=(15, 15))
plt.subplot(3, 1, 1)
plt.errorbar(xx, yyI, fmt='-or', label='QUBIC acquisition only')
plt.errorbar(xx_pl, yyI_pl, fmt=':ob', label='QUBIC + Planck acquisition')
plt.axhline(np.std(noise_planck[:, 0]), color='black', ls='--')
plt.yscale('log')
plt.xlabel('')
plt.ylabel(r'RMS [$\mu K$]', fontsize=15)
plt.legend(frameon=False, fontsize=15)
plt.title('I', fontsize=15)
plt.ylim(4e-1, 5e2)

plt.subplot(3, 1, 2)
plt.errorbar(xx, yyQ, fmt='-or')
plt.errorbar(xx_pl, yyQ_pl, fmt=':ob')
plt.axhline(np.std(noise_planck[:, 1]), color='black', ls='--')
plt.yscale('log')
plt.xlabel('')
plt.ylabel(r'RMS [$\mu K$]', fontsize=15)
plt.title('Q', fontsize=15)
plt.ylim(4e-1, 5e2)

plt.subplot(3, 1, 3)
plt.errorbar(xx, yyU, fmt='-or')
plt.errorbar(xx_pl, yyU_pl, fmt=':ob')
plt.axhline(np.std(noise_planck[:, 2]), color='black', ls='--')
plt.yscale('log')
plt.xlabel(r'$\theta$ [°]', fontsize=15)
plt.ylabel(r'RMS [$\mu K$]', fontsize=15)
plt.title('U', fontsize=15)
plt.ylim(4e-1, 5e2)
plt.show()